In [1]:
%pip install streamlit -q
%pip install python-dotenv -q
%pip install Pillow -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%%writefile app.py

import streamlit as st
import google.generativeai as genai
import dotenv
import os
from PIL import Image

dotenv.load_dotenv()
api_key = os.getenv("API_KEY")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash-exp-image-generation")


SYSTEM_PROMPT = """You are a witty product-safety assistant! I will give you product names. For each one:

1.  Check if the product contains harmful ingredients. If yes, list them.
2.  Give a recommendation score from 1 to 5 (1 = Avoid, 5 = Safe & Awesome).
3.  Provide links to the product from 2 different e-commerce websites (e.g., Walmart, Target). Include the product name in the URL if possible. State the current price next to each link.
4.  Suggest keywords that could be used to find a relevant image for this product online.
5.  Keep your response under 150 words per product. Make it informative but fun with emoticons!"""


def analyze_product(product_name):
    prompt = f"""Analyze the safety of the following product: {product_name}
            Provide information on harmful ingredients, a safety score (1-5), and links to page with that product online (with prices). Also, suggest image keywords."""
    try:
        response = model.generate_content(
            [{"role": "user", "parts": SYSTEM_PROMPT}, {"role": "user", "parts": prompt}]
        )
        return response.candidates[0].content.parts[0].text
    except Exception as e:
        return f"Uh oh! Something went wrong: {str(e)}"


st.title("Safety Sleuth 🕵️‍♀️ - Your Witty Product Pal")

if "messages" not in st.session_state:
    st.session_state["messages"] = []

user_input = st.chat_input("Enter a product name:")

if user_input:
    st.session_state["messages"].append({"role": "user", "parts": user_input})
    safety_analysis = analyze_product(user_input)
    st.session_state["messages"].append({"role": "model", "parts": safety_analysis})

for message in st.session_state["messages"]:
    if message["role"] == "model":
        st.markdown(f"**Safety Sleuth:** {message['parts']}", unsafe_allow_html=True)
    elif message["role"] == "user":
        st.write(f"**You:** {message['parts']}")

Overwriting app.py


In [ ]:
%streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com
%npx localtunnel --port 8501